In [1]:
## import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#ing "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pubg-finish-placement-prediction/train_V2.csv
/kaggle/input/pubg-finish-placement-prediction/test_V2.csv
/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv


In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/train_V2.csv")
test = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/test_V2.csv")

In [4]:
train.dropna(axis = 0, inplace =True)

In [5]:
label_name = 'winPlacePerc'
X_tr_all = train.loc[:,['assists','kills','killPoints','DBNOs','killStreaks','boosts','revives','heals','matchType']]
train = train.astype({'winPlacePerc':'int'})
y_tr_all = train[label_name]

X_last_test = test.loc[:,['assists','kills','killPoints','DBNOs','killStreaks','boosts','revives','heals','matchType']]

In [6]:
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'solo-fpp':11})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'duo-fpp':21})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'solo':13})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'duo':23})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'squad':43})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'squad-fpp':41})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-solo-fpp':110})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-duo-fpp': 210})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-squad-fpp':410})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-solo':130})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-duo':230})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-squad':430})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'crashfpp':10})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'crashtpp':30})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'flarefpp':19})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'flaretpp':39})
X_tr_all['matchType']

0          41
1          41
2          23
3          41
4          11
           ..
4446961    41
4446962    13
4446963    41
4446964    41
4446965    11
Name: matchType, Length: 4446965, dtype: int64

In [7]:
X_tr_all['matchType'].unique()

array([ 41,  23,  11,  43,  21,  13, 410,  10,  39, 110,  19, 210, 230,
       430,  30, 130])

In [8]:
X_last_test.columns

Index(['assists', 'kills', 'killPoints', 'DBNOs', 'killStreaks', 'boosts',
       'revives', 'heals', 'matchType'],
      dtype='object')

In [9]:
X_last_test['matchType'] = X_last_test['matchType'].replace({'solo-fpp':11})
X_last_test['matchType'] = X_last_test['matchType'].replace({'duo-fpp':21})
X_last_test['matchType'] = X_last_test['matchType'].replace({'solo':13})
X_last_test['matchType'] = X_last_test['matchType'].replace({'duo':23})
X_last_test['matchType'] = X_last_test['matchType'].replace({'squad':43})
X_last_test['matchType'] = X_last_test['matchType'].replace({'squad-fpp':41})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-solo-fpp':110})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-duo-fpp': 210})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-squad-fpp':410})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-solo':130})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-duo':230})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-squad':430})
X_last_test['matchType'] = X_last_test['matchType'].replace({'crashfpp':10})
X_last_test['matchType'] = X_last_test['matchType'].replace({'crashtpp':30})
X_last_test['matchType'] = X_last_test['matchType'].replace({'flarefpp':19})
X_last_test['matchType'] = X_last_test['matchType'].replace({'flaretpp':39})
X_last_test['matchType']

0          41
1          21
2          41
3          21
4          41
           ..
1934169    41
1934170    41
1934171    41
1934172    21
1934173    41
Name: matchType, Length: 1934174, dtype: int64

In [10]:
X_last_test['matchType'].unique()

array([ 41,  21,  43,  11,  13,  23, 410,  30, 210,  10, 230,  39, 110,
       130,  19, 430])

In [11]:
X_last_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934174 entries, 0 to 1934173
Data columns (total 9 columns):
 #   Column       Dtype
---  ------       -----
 0   assists      int64
 1   kills        int64
 2   killPoints   int64
 3   DBNOs        int64
 4   killStreaks  int64
 5   boosts       int64
 6   revives      int64
 7   heals        int64
 8   matchType    int64
dtypes: int64(9)
memory usage: 132.8 MB


In [12]:
X_tr_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Data columns (total 9 columns):
 #   Column       Dtype
---  ------       -----
 0   assists      int64
 1   kills        int64
 2   killPoints   int64
 3   DBNOs        int64
 4   killStreaks  int64
 5   boosts       int64
 6   revives      int64
 7   heals        int64
 8   matchType    int64
dtypes: int64(9)
memory usage: 339.3 MB


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X_tr_all, y_tr_all, test_size = 0.3, random_state = 33)

In [14]:
model = LGBMClassifier()
model = model.fit(X_train,y_train)

In [15]:
pred = model.predict_proba(X_last_test)[:,1]
pred

array([0.00302068, 0.09953728, 0.01055942, ..., 0.01122309, 0.02228706,
       0.00288612])

In [16]:
len(pred)

1934174

In [17]:
sample_submission = pd.read_csv("../input/pubg-finish-placement-prediction/sample_submission_V2.csv")
sample_submission.head()

,Id,winPlacePerc
0,9329eb41e215eb,1
1,639bd0dcd7bda8,1
2,63d5c8ef8dfe91,1
3,cf5b81422591d1,1
4,ee6a295187ba21,1


In [18]:
sample_submission.shape

(1934174, 2)

In [19]:
sample_submission["winPlacePerc"] = pred
sample_submission.to_csv("submission.csv", index=False)